# Контрольная работа №1: Линейная и полиномиальная регрессия

**ФИО: Приемышев Илья Александрович**

**Группа: ИД23-2**

**Вариант: 6** 

Импортируем необходимые библиотеки:

In [31]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.datasets import fetch_openml

### 1. Загрузите предложенный вам датасет из библиотеки sklearn.datasets.fetch_openml. Выведете описание загруженного датасета. Обозначьте матрицу признаков за X, а целевую переменную за y. Убедитесь, что в данных нет пропусков и они числовые. Выведите их размеры.

В качестве ответа к заданию укажите количество столбцов в предложенном датасете.

In [92]:
data = fetch_openml('kin8nm', version=1)

In [93]:
print(data.DESCR)

**Author**:   
**Source**: Unknown -   
**Please cite**:   

This is data set is concerned with the forward kinematics of an 8 link
 robot arm. Among the existing variants of this data set we have used
 the variant 8nm, which is known to be highly non-linear and medium
 noisy.

 Original source: DELVE repository of data. 
 Source: collection of regression datasets by Luis Torgo (ltorgo@ncc.up.pt) at
 http://www.ncc.up.pt/~ltorgo/Regression/DataSets.html
 Characteristics: 8192 cases, 9 attributes (0 nominal, 9 continuous).

Downloaded from openml.org.


In [94]:
X = data.data
y = data.target

In [95]:
X.isnull().values.any(), y.isnull().values.any(), all(X.dtypes.apply(lambda x: pd.api.types.is_numeric_dtype(x)))

(False, False, True)

In [96]:
X.shape, y.shape

((8192, 8), (8192,))

### Ответ к заданию 1: 

In [97]:
f'{X.shape[1]} - количество столбцов в предложенном датасете'

'8 - количество столбцов в предложенно датасете'

### 2. Разделите данные на обучающую и тестовую части с перемешиванием, выделив в тестовую часть 20% объектов, а также зафиксируйте random_state=42 для воспроизводимости разбиения.

В качестве ответа к заданию укажите количество объектов, попавших в обучающий набор.

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### Ответ к заданию 2:

In [99]:
f'{X_train.shape[0]} - количество объектов, попавших в обучающий набор'

'6553 - количество объектов, попавших в обучающий набор'

### 3. Выполните масштабирование признаков методом стандартной нормализации.
В качестве ответа к заданию укажите среднее значение по всей матрице масштабированных тестовых данных с точностью три знака после запятой.

In [100]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [103]:
mean_test_scaled = round(X_test_scaled.mean(), 3)

### Ответ к заданию 3:

In [104]:
f'{mean_test_scaled} - среднее значение по всей матрице масштабированных тестовых данных с точностью три знака после запятой'

'-0.004 - среднее значение по всей матрице масштабированных тестовых данных с точностью три знака после запятой'

### 4. Постройте на масштабированных данных модели линейной регрессии из библиотеки sklearn и написанную своими руками методом градиентного спуска с количеством шагов 1000 и скоростью обучения 0.15. Выведите, полученные коэффициенты  гиперплоскости. Убедитесь, что полученные коэффициенты приблизительно совпадают.
В качестве ответа к заданию укажите значение свободного коэффициента с точностью три знака после запятой.

In [105]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [106]:
model.coef_, model.intercept_

(array([-0.0353803 , -0.02251093, -0.13871613, -0.0234454 ,  0.065472  ,
        -0.03756655, -0.03830163,  0.01732101]),
 0.7150037555124372)

In [45]:
class Linear_Regression_GD():
    def __init__(self, max_iter):
        self.w = None
        self.max_iter_ = max_iter

    def X_(self, X):
        X_ones = np.ones((X.shape[0], 1))
        return np.hstack([X_ones, X])

    def predict(self, X):
        return np.dot(self.X_(X), self.w)

    def gradient_F(self, X, y):
        return (2/X.shape[0])*self.X_(X).transpose()@(self.predict(X)-y)

    def GD(self, a, b, w_start, learning_rate = 0.15):
        self.w = w_start

        for _ in range(self.max_iter_):
            self.w = self.w - learning_rate *self.gradient_F(a, b)
        assert (-1e+06 < self.w).all() and (self.w < 1e+06).all(), "Расходимость: слишком большой learning_rate, либо неудачно выбрана начальная точка, либо минимум не достигается"
        assert (-1e-04 < self.gradient_F(a, b)).all() and (self.gradient_F(a, b) < 1e-04).all(), "Недостаточно шагов градиентного спуска"

    def fit(self, a, b):
        w_start = np.zeros(a.shape[1] + 1)
        self.GD(a, b, w_start)

In [47]:
lr = Linear_Regression_GD(1000)
lr.fit(X_train_scaled, y_train)

In [48]:
lr.w

array([ 0.71500376, -0.0353803 , -0.02251093, -0.13871613, -0.0234454 ,
        0.065472  , -0.03756655, -0.03830163,  0.01732101])

### Ответ к заданию 4:

In [56]:
f'{round(model.intercept_, 3)} - значение свободного коэффициента с точностью три знака после запятой'

'0.715 - значение свободного коэффициента с точностью три знака после запятой'

### 5. Сделайте предсказание на масштабированных тестовых данных для модели из библиотеки sklearn. Вычислите метрики: коэффициент детерминации и ошибка RMSE. Сделайте вывод о качестве работы модели линейной регрессии для этих данных. 
В качестве ответа к заданию укажите значение метрики RMSE с точностью три знака после запятой.

In [107]:
y_pred = model.predict(X_test_scaled)

In [108]:
r2 = r2_score(y_test, y_pred)
r2

0.4083491705257649

In [109]:
rmse = root_mean_squared_error(y_test, y_pred)
rmse

0.2023306590137644

### Ответ к заданию 5:

In [110]:
round(rmse, 3)

0.202

### 6. Постройте модель полиномиальной регрессии второй степени, используя модель линейной регрессии из библиотеки sklearn. Вычислите метрики качества: коэффициент детерминации и ошибку RMSE. Сделайте вывод о том, какая из моделей линейная или полиномиальная регрессия описывает данные лучше. 
Указание: Из тренировочных данных получите полиномиальные данные 2 степени, а из тестовых данных получите полиномиальные тестовые данные аналогичным образом. Масштабируйте полученные данные и обучите модель линейной регрессии на масштабированных обучающих данных. Сделайте предсказание на масштабированных тестовых данных и вычислите метрики качества. 

В качестве ответа к заданию укажите значение коэффициента детерминации с точностью три знака после запятой.

In [111]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.transform(X_test)

In [112]:
scaler = StandardScaler()
X_poly_train_scaled = scaler.fit_transform(X_poly_train)
X_poly_test_scaled = scaler.transform(X_poly_test)

In [113]:
model_poly = LinearRegression()
model_poly.fit(X_poly_train_scaled, y_train)

LinearRegression()

In [114]:
y_pred_poly = model_poly.predict(X_poly_test_scaled)

In [115]:
r2_poly = r2_score(y_test, y_pred_poly)
r2_poly

0.5552536779373064

In [116]:
rmse_poly = root_mean_squared_error(y_test, y_pred_poly)
rmse_poly

0.17542243693513404

### Ответ к заданию 6:

In [117]:
f'{round(r2_poly, 3)} - значение коэффициента детерминации с точностью три знака после запятой'

'0.555 - значение коэффициента детерминации с точностью три знака после запятой'